# Chapter 11: Testing with Out-of-Sample Data

In [1]:
import sys
import os
sys.path.append('/home/yutanagano/Projects/nnfs')
os.chdir('/home/yutanagano/Projects/nnfs')

In [2]:
import numpy as np
import nnfs
from nnfs.datasets import spiral_data

nnfs.init()

import nnn

## Setup

In [3]:
# Create dataset
X, y = spiral_data(samples=100, classes=2)
y = y.reshape(-1, 1)

# Create the necessary layers
dense1 = nnn.layer.Dense(n_inputs=2,n_neurons=64,l2w=5e-4,l2b=5e-4)
activation1 = nnn.activation.Relu()
dense2 = nnn.layer.Dense(n_inputs=64,n_neurons=1)
activation2 = nnn.activation.Sigmoid()

loss_fn = nnn.loss.BinaryCrossentropy()

# Create the optimiser
optimiser = nnn.optimiser.Adam(decay=5e-7)

## Training

In [4]:
# Training loop
for epoch in range(15001):
    # Forward pass
    output = dense1.forward(X)
    output = activation1.forward(output)
    output = dense2.forward(output)
    output = activation2.forward(output)

    # Calculate the network's current loss
    data_loss = loss_fn.calculate(output, y)
    reg_loss = loss_fn.regularsiation_loss(dense1) +\
        loss_fn.regularsiation_loss(dense2)
    loss = data_loss + reg_loss

    # Calculate accuracy
    predictions = (activation2.outputs > 0.5) * 1
    accuracy = np.mean(predictions == y)

    # Print accuracy
    if not epoch % 100: print(
        f"epoch: {epoch}, acc: {accuracy:.3f}, loss: {loss:.3f}, "
        f"(data_loss: {data_loss}, reg_loss: {reg_loss}), "
        f"lr: {optimiser.current_learning_rate}"
    )

    # Backward pass
    loss_fn.backward(activation2.outputs, y)
    activation2.backward(loss_fn.dinputs)
    dense2.backward(activation2.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    # Update weights and biases
    optimiser.pre_update_params()
    optimiser.update_params(dense1)
    optimiser.update_params(dense2)
    optimiser.post_update_params()

epoch: 0, acc: 0.500, loss: 0.693, (data_loss: 0.6931493878364563, reg_loss: 5.625688005238772e-06), lr: 0.001
epoch: 100, acc: 0.620, loss: 0.626, (data_loss: 0.6185082197189331, reg_loss: 0.007556769467890263), lr: 0.0009999505024501287
epoch: 200, acc: 0.750, loss: 0.558, (data_loss: 0.5476176738739014, reg_loss: 0.010370809972286225), lr: 0.0009999005098992651
epoch: 300, acc: 0.785, loss: 0.500, (data_loss: 0.4877138137817383, reg_loss: 0.012202283293008804), lr: 0.000999850522346909
epoch: 400, acc: 0.815, loss: 0.459, (data_loss: 0.44591981172561646, reg_loss: 0.013429276823997497), lr: 0.0009998005397923115
epoch: 500, acc: 0.825, loss: 0.434, (data_loss: 0.41967594623565674, reg_loss: 0.014222035169601441), lr: 0.0009997505622347225
epoch: 600, acc: 0.815, loss: 0.418, (data_loss: 0.4029194116592407, reg_loss: 0.01475272810459137), lr: 0.0009997005896733929
epoch: 700, acc: 0.840, loss: 0.405, (data_loss: 0.3897309899330139, reg_loss: 0.01506212306022644), lr: 0.00099965062210

## Evaluation

In [5]:
# Create the validation data set
X_test, y_test = spiral_data(samples=100, classes=2)
y_test = y_test.reshape(-1, 1)

In [6]:
# Forward pass on validation set
output = dense1.forward(X_test)
output = activation1.forward(output)
output = dense2.forward(output)
output = activation2.forward(output)

# Calculate validation loss
loss = loss_fn.calculate(output, y_test)

In [7]:
# Calculate accuracy
predictions = (activation2.outputs > 0.5) * 1
accuracy = np.mean(predictions == y_test)

print(f"validation, acc: {accuracy:.3f}, loss: {loss:.3f}")

validation, acc: 0.895, loss: 0.353
